In [5]:
from database import *
def day_trading_backtest(code, date, connection, api):
    ticks= get_ticks(code, date, connection, api)[0]
    kbars = ticks_to_kbars(ticks)
    
    entry_time = None
    entry_price = 0
    
    exit_time = None
    exit_price = 0
    
    position = 0
    
    for ts in range(len(kbars)):
        current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=1)
        current_price = kbars['close'][ts]
        
        if (
            current_time == date.replace(hour=9, minute=30, second=0) and
            position == 0
        ):
            
            position = 1
            entry_time = current_time.time()
            entry_price = current_price
            
            print('[{}] buy {} at {}'.format(current_time, code, current_price))
            
        elif (
            current_time == date.replace(hour=13, minute=0, second=0) and
            position != 0
         ):
            
            exit_time = current_time.time()
            exit_price = current_price
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
    if entry_time and exit_time:
        transaction = pd.DataFrame([[date, 
                                    code, 
                                    entry_time,
                                    entry_price, 
                                    position * 1000,
                                    entry_price * position * 1000,
                                    exit_time,
                                    exit_price,
                                    position * 1000,
                                    exit_price * position * 1000]
                                       ],
                                        columns=[
                                           '成交日期', 
                                            '股票代號',
                                            '買進時間',
                                            '買進價格',
                                            '買進股數',
                                            '買進金額',
                                            '賣出時間',
                                            '賣出價格',
                                            '賣出股數',
                                            '賣出金額'])
        return transaction
    else:
        return pd.DataFrame()

In [6]:
import shioaji as sj
api = sj.Shioaji()
api.login(
    person_id='N123569782',
    passwd='hons3146',
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


Exception: {'status': {'status_code': 400}, 'response': {'token': '', 'accounts': [], 'detail': 'ORA-20006:   passwd_err_3_times\nORA-06512: at "STSTX.SGTP_LOGIN_G", line 407\nORA-06512: at line 1', 'ip_address': '223.141.186.131', 'client_name': ''}}

In [8]:
day_trading_backtest('2441', pd.to_datetime('2021/6/16'), connection, api)

AttributeError: 'NoneType' object has no attribute 'dict'